# json brut à CSV
### Création CSV to_eval, liste de IPC secondaire pour ES

In [1]:
import os
os.chdir('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP')
print(os.getcwd())

C:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP


In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm


In [21]:
mypath = "data/2008/"  #'data/2016/2016/'
_2009_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [22]:
print(len(_2009_files))
print(_2009_files[1])
patent_path = mypath + _2009_files[9]
print(patent_path)
with open(patent_path) as f:
    d = json.load(f) # load json in d
    f.close()   # close f - not needed with with ___ as syntax
class_mainIPC = d['main_ipcr_label']
print(d)
print(d["main_ipcr_label"])

for i in tqdm(range(10)):
    patent_path = mypath + _2009_files[i]
    with open(patent_path) as f:
        d = json.load(f) # load json in d
        f.close()   # close f - not needed with with ___ as syntax
    class_mainIPC = d['main_ipcr_label']
    secClass = d['ipcr_labels']
    if len(secClass) > 1:
        print(i)

    


314526
10473685.json
data/2008/10514296.json
{'application_number': '10514296', 'publication_number': 'US20090260355A1-20091022', 'title': 'Stirling engine assembly', 'decision': 'REJECTED', 'date_produced': '20091007', 'date_published': '20091022', 'main_cpc_label': '', 'cpc_labels': [], 'main_ipcr_label': 'F02G104', 'ipcr_labels': ['F02G104'], 'patent_number': 'None', 'filing_date': '20080107', 'patent_issue_date': '', 'abandon_date': '20110331', 'uspc_class': '60', 'uspc_subclass': '520000', 'examiner_id': '71796.0', 'examiner_name_last': 'JETTON', 'examiner_name_first': 'CHRISTOPHER', 'examiner_name_middle': '', 'inventor_list': [{'inventor_name_last': 'Alderson', 'inventor_name_first': 'John Howard', 'inventor_city': 'Matlock', 'inventor_state': '', 'inventor_country': 'GB'}, {'inventor_name_last': 'Clark', 'inventor_name_first': 'David Anthony', 'inventor_city': 'Hugglescote', 'inventor_state': '', 'inventor_country': 'GB'}, {'inventor_name_last': 'Hyde', 'inventor_name_first': '

 50%|█████     | 5/10 [00:00<00:00, 45.28it/s]

0
1
3


100%|██████████| 10/10 [00:00<00:00, 53.30it/s]

8


In [23]:
# Création de listes de brevets de chaque type de class IPC à préciser

# Check on all decision and repartition -- I think we should keep only the one accepted based on other USPTO patent (continuity)

# initialisation de dict, dont les valeurs inconnues sont initialiser à 0, pour chaque classe IPC
decision_H01L = collections.defaultdict(int)
decision_G06F = collections.defaultdict(int)
decision_A61B = collections.defaultdict(int)
nbEmpty = collections.defaultdict(int)

# initialisation de vecteurs pour chaque classe IPC
patent_H01L_list = []
patent_G06F_list = []
patent_A61B_list = []

for i in tqdm(range(len(_2009_files[:10000]))):
    patent_path = mypath + _2009_files[i]
    with open(patent_path) as f:
        d = json.load(f) # load json in d
        f.close()   # close f - not needed with "with ___ as" syntax
    class_mainIPC = d['main_ipcr_label']
    if re.match(r'^H01L', class_mainIPC):
        patent_H01L_list.append(_2009_files[i])
        decision_H01L[d['decision']] += 1
    if re.match(r'^G06F', class_mainIPC):
        patent_G06F_list.append(_2009_files[i])
        decision_G06F[d['decision']] += 1
    if re.match(r'^A61B', class_mainIPC):
        patent_A61B_list.append(_2009_files[i])
        decision_A61B[d['decision']] += 1
    if class_mainIPC == "":
        nbEmpty[class_mainIPC] += 1

100%|██████████| 10000/10000 [03:01<00:00, 55.19it/s]


In [20]:
print(len(patent_H01L_list), len(patent_G06F_list), len(patent_A61B_list))
print(decision_H01L, decision_G06F, decision_A61B)
print(nbEmpty)

236 432 234
defaultdict(<class 'int'>, {'ACCEPTED': 148, 'REJECTED': 87, 'PENDING': 1}) defaultdict(<class 'int'>, {'REJECTED': 222, 'ACCEPTED': 207, 'PENDING': 3}) defaultdict(<class 'int'>, {'REJECTED': 122, 'ACCEPTED': 109, 'PENDING': 3})
defaultdict(<class 'int'>, {'': 1})


In [24]:
print(nbEmpty)

defaultdict(<class 'int'>, {})


Il y a énormément de brevets sans IPC...

Surtout en 2004, mais aussi _2009

- 10018639 - _20090081931A1
- 10018320 - _20090130116A1
- 10250708 - 20060086443A1

Nous allons continuer avec 2006 ici

#### Exclusion des de tout autre que Accepter ou rejeter, dans une liste de brevet de classe IPC

In [110]:
final_patents = []
decision = []
for i in tqdm(range(len(patent_A61B_list))):
    patent_path = mypath + patent_A61B_list[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    if d['decision'] == 'ACCEPTED' or d['decision'] == 'REJECTED': # exclure pending
        final_patents.append(patent_A61B_list[i])
        decision.append(d['decision'])

100%|██████████| 242/242 [00:00<00:00, 3893.54it/s]


In [34]:
print(final_patents)
print(decision)

['10484186.json', '10493926.json', '10505531.json', '10511495.json', '10512150.json', '10512772.json', '10513656.json', '10516287.json', '10518848.json', '10518867.json', '10519023.json', '10520965.json', '10521373.json', '10522197.json', '10522609.json', '10524196.json', '10524984.json', '10526112.json', '10527781.json', '10528417.json', '10528504.json', '10528841.json', '10528922.json', '10529150.json', '10529770.json', '10530094.json', '10530811.json', '10530860.json', '10531204.json', '10531378.json', '10531764.json', '10531819.json', '10531832.json', '10531958.json', '10532691.json', '10532704.json', '10533596.json', '10534268.json', '10535214.json', '10535227.json', '10535316.json', '10535617.json', '10535726.json', '10535730.json', '10535775.json', '10535914.json', '10535958.json', '10536480.json', '10536503.json', '10536605.json', '10537103.json', '10537235.json', '10537795.json', '10538155.json', '10538411.json', '10538992.json', '10539175.json', '10539255.json', '10539818.jso

#### Création CSV à partir de list de json (1 par classe IPC et par année)

Infos à extraire: \
- numéro de brevet
- informations textuelles :
    - Anciens
        - ID
        - titre
        - background
        - claims
        - Décision
    - Ajout:
        - Abstract (pourquoi on ajoute?)
        - Summary (pourquoi on ajoute?)
        - IPC principale (pour seconde analyse)
        - IPC secondaire (pour expectation space, pas directement dans CSV)


In [111]:
non_main_ipc = []
labels = []
patent_number, titles, backgrounds, claims, summary, abstract, ipc = [], [], [], [], [], [], []
for i in tqdm(range(len(final_patents))):
    patent_path = mypath + final_patents[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close() # ligne inutile
    
    #Creating the lists for the other information
    patent_number.append(d['application_number'])
    titles.append(d['title'])
    backgrounds.append(d['background'])
    claims.append(d['claims'])
    summary.append(d['summary'])
    abstract.append(d['abstract'])
    ipc.append(d['main_ipcr_label'])

    #Collecting non main ipc class
    non_main =  d['ipcr_labels']
    for ipcr in non_main:
        non_main_ipc.append(ipcr) # only 4 first characters to be sure of being at same level
    #Getting labels based on decision
    label = 0
    if d['decision'] == 'ACCEPTED':
        label = 1
    labels.append(label)

100%|██████████| 241/241 [00:00<00:00, 2985.72it/s]


only 4 first characters to be sure of being at same level \
Getting labels based on decision????

In [50]:
pd.options.display.max_colwidth
pd.options.display.max_rows

60

In [112]:
print(len(non_main_ipc))
print(patent_number)
print(non_main_ipc)
print(claims[0])
print(len(ipc))
print(len(labels))

383
['10513457', '10551443', '10574242', '10585412', '10585430', '10585439', '10585494', '10585500', '10585565', '10595493', '10595672', '11547559', '11568603', '11571511', '11571611', '11571871', '11573493', '11576229', '11576321', '11597205', '11630235', '11660466', '11660965', '11664172', '11665122', '11665587', '11718301', '11719207', '11719400', '11719484', '11719585', '11719720', '11721287', '11721327', '11721445', '11721749', '11721993', '11722079', '11791448', '11792482', '11794650', '11795281', '11814765', '11815247', '11884684', '11885899', '11885938', '11886911', '11886934', '11887434', '11887582', '11887640', '11908073', '11908648', '11909815', '11910411', '11910691', '11911111', '11911479', '11911606', '11912678', '11913317', '11913372', '11913852', '11914497', '11914814', '11915102', '11915468', '11915625', '11915642', '11915884', '11916168', '11917318', '11917787', '11918145', '11918342', '11918347', '11918807', '11918988', '11919815', '11919851', '11920140', '11920314',

In [113]:
expectations_classes = list(set(non_main_ipc)) # création d'éléments unique - 253 à 76 éléments
print(expectations_classes)
print(len(expectations_classes))

['A61B173203', 'B23P1304', 'G01N3348', 'A61N130', 'B02C2300', 'G06F1730', 'A61B500', 'A61K3836', 'G02B2106', 'A61F500', 'A61B1802', 'G01R3332', 'A61F1320', 'A61B5053', 'G01N2306', 'A61B3113', 'A61B1900', 'B32B3700', 'A61B5021', 'A61B17068', 'A61B51477', 'A61B173211', 'A61B1247', 'A61K900', 'A61M2500', 'A61B1743', 'A61B1822', 'A61B604', 'A61K31167', 'G06G758', 'A61B1764', 'A61B1012', 'A61N100', 'A61B1814', 'A61B1708', 'A61B101', 'A61C1706', 'A61B316', 'A61K3324', 'A61B1902', 'A61B1908', 'B32B502', 'A61B17225', 'A61C1310', 'A61B50488', 'A61B50402', 'A61M3100', 'A61B5085', 'A61B806', 'B05D300', 'G01N2300', 'G06K900', 'H01L2966', 'A61M2510', 'A61F202', 'A61B132', 'C04B35495', 'A61B800', 'A61B1722', 'G01S1300', 'A61B511', 'B29D1100', 'A61B812', 'A61B1018', 'A61B1002', 'A61B50205', 'A61B1704', 'H01Q2100', 'A61B1045', 'H05B3900', 'A61H102', 'A61B614', 'G09B1900', 'H05G102', 'A61B507', 'A61K4702', 'G06K1906', 'A61B1780', 'H04B106', 'A61B504', 'A61B173205', 'A61B1005', 'A61B813', 'A61B1728', 'A

In [114]:
# Retirer les classes secondaires de la même classe que la princiaple
good_expectations_classes = []
for ipcr in expectations_classes:
    if ipcr[0:4] != "A61B":
        good_expectations_classes.append(ipcr)
        
print(len(good_expectations_classes))
print(good_expectations_classes)

82
['B23P1304', 'G01N3348', 'A61N130', 'B02C2300', 'G06F1730', 'A61K3836', 'G02B2106', 'A61F500', 'G01R3332', 'A61F1320', 'G01N2306', 'B32B3700', 'A61K900', 'A61M2500', 'A61K31167', 'G06G758', 'A61N100', 'A61C1706', 'A61K3324', 'B32B502', 'A61C1310', 'A61M3100', 'B05D300', 'G01N2300', 'G06K900', 'H01L2966', 'A61M2510', 'A61F202', 'C04B35495', 'G01S1300', 'B29D1100', 'H01Q2100', 'H05B3900', 'A61H102', 'G09B1900', 'H05G102', 'A61K4702', 'G06K1906', 'H04B106', 'G06N504', 'G01B1106', 'B32B106', 'G06F1518', 'G06T100', 'G06K708', 'B29C5112', 'G06Q1000', 'H04N718', 'A61F1100', 'A61L1516', 'H01Q142', 'B05D302', 'A61M3700', 'A61K4918', 'A01K67027', 'F16L3237', 'A61M1600', 'B65B508', 'F21L400', 'B32B900', 'C12N1512', 'A61N700', 'G06Q5000', 'B32B2702', 'G06Q3000', 'G01R3344', 'G06F1900', 'G02B600', 'B32B108', 'C07D48700', 'G06K700', 'B23P1704', 'C12N510', 'C12N1585', 'G08B500', 'H04M100', 'G06T1500', 'A61F244', 'B29C3300', 'A61F206', 'H05G100', 'A61M500']


In [117]:
# Sauvegarder IPC secondaire en format texte
with open(r'exemple données/expectation_space/2009_A61B_expectation_IPC_class.txt', 'w') as fp:
    for item in good_expectations_classes:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [116]:
# Création jdD à mettre en csv
df = pd.DataFrame({'application_number': patent_number, 'title': titles, 'abstract':abstract,
                    'claims':claims, 'background': backgrounds, 'summary':summary, 'ipc':ipc, 'label': labels})
df  

,application_number,title,abstract,claims,background,summary,ipc,label
0,10513457,Surgical thread and surgical implant with the ...,A surgical thread (10) has a first component (...,1. Surgical thread with a first component (12;...,,,A61B1704,0
1,10551443,Device and method for determining the orientat...,In a device or a method for determining the di...,1-12. (canceled) 13. A device for determining ...,<SOH> BACKGROUND OF THE INVENTION <EOH>1. Fiel...,<SOH> SUMMARY OF THE INVENTION <EOH>This objec...,A61B3113,1
2,10574242,Method and apparatus for an improved sample ca...,A body fluid sampling device is provided. A me...,1. A body fluid sampling device comprising: a ...,<SOH> BACKGROUND OF THE INVENTION <EOH>Lancing...,<SOH> SUMMARY OF THE INVENTION <EOH>The presen...,A61B500,1
3,10585412,TROCAR DEVICE FOR PASSING A SURGICAL TOOL,This invention relates to a trocar device (9) ...,1. Trocar device (9) for passing a surgical in...,,,A61B1734,0
4,10585430,IMPLANTABLE DEVICE DELIVERY SYSTEM HANDLE AND ...,"The present invention, in an exemplary embodim...",1. A device for allowing a user to deploy a st...,<SOH> BACKGROUND OF THE INVENTION <EOH>Interve...,<SOH> SUMMARY OF EXEMPLARY EMBODIMENTS <EOH>Ac...,A61B1900,1
...,...,...,...,...,...,...,...,...
236,12293145,COMPRESSED AIR DISSECTOR (AIR JET SCRAPER),Compressed air dissector is a surgical dissect...,1-9. (canceled) 10. A system for scraping of t...,<SOH> TECHNICAL FIELD <EOH>Subject invention i...,,A61B173203,1
237,12293233,PULSE-WAVE MEASUREMENT APPARATUS,A pulse-wave measurement apparatus includes a ...,1. A pulse-wave measurement apparatus for meas...,<SOH> BACKGROUND OF THE INVENTION <EOH>As a pr...,<SOH> SUMMARY OF THE INVENTION <EOH>On the oth...,A61B502,0
238,12293829,CANNULA EQUIPPED WITH A PROTECTIVE HOUSING,"The invention relates to a cannula, preferably...",1-2. (canceled) 3. A cannula equipped with a c...,,,A61B515,1
239,12293932,Vascular Anastomotic Staplers,"Vascular anastomotic staplers (100, 200, 300) ...",1. A stapler for simultaneously discharging a ...,<SOH> BACKGROUND OF THE INVENTION <EOH>It is w...,<SOH> SUMMARY OF THE INVENTION <EOH>The presen...,A61B17068,0


In [118]:
# Sauvegarder CSV to_eval
df.to_csv('exemple données/to_eval/2014_A61B_patents_toeval.csv', index=False)